*This is currently still a draft which means it may be messy to read!*

Below is a short demonstration on using AutoTokenizer from [transformers](https://pypi.org/project/transformers/) package to tokenise/encode and decode one line of texts. A separate notebook (5-1_Tokenizers_to_tokenise_texts.ipynb) has been prepared to show how to use tokenizers package to tokenise/encode and decode text data or ADR terms.

Reference links: 
* https://huggingface.co/learn/llm-course/chapter2/4?fw=pt#tokenization
* https://huggingface.co/docs/datasets/use_dataset#tokenize-text

Plan
* initial small goal is to try using tokenizer.decode(), so building a tokenization/tokenizer model first
* tokenisation appears to be workable through two different packages e.g. tokenizers or transformers package
* the idea is that this text data tokenisation part can be added to a larger DNN model to decode ADRs output later (subject to further idea changes... may try a small NER classification model first, see 6_NER_tk_inhibitors.ipynb)

* trying HuggingFace's transformers package:
1. set up tokenizer model that will tokenize the ADRs/words
2. apply tokenizer.decode() function to each tensor row/sequence (via using list comprehension)
3. use sample code snippet below to decode tensors: 
e.g. decoded = [tokenizer.decode(x) for x in adrs_ts]
- the code will iterate through each row/sequence of tensors and apply the decode() method 
which'll transform the numerical IDs back into human-readable texts/words

In [1]:
# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.nn.functional import one_hot
# from torch.utils.data import TensorDataset, DataLoader
# import numpy as np
# import datamol as dm
# import rdkit
# from rdkit import Chem
# from rdkit.Chem import rdFingerprintGenerator
# import useful_rdkit_utils as uru
# from matplotlib import pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import sys, datetime

# print(f"Pandas version used: {pd.__version__}")
# print(f"PyTorch version used: {torch.__version__}")
# print(f"NumPy version used: {np.__version__}")
#print(f"RDKit version used: {rdkit.__version__}")
print(f"Python version used: {sys.version} at {datetime.datetime.now()}")

Python version used: 3.12.7 (main, Oct 16 2024, 09:10:10) [Clang 18.1.8 ] at 2025-05-20 14:24:19.085140


In [2]:
# PyTorch example re. saving & reloading tensors
# t = torch.tensor([1., 2.])
# torch.save(t, 'tensor.pt')
# ts = torch.load('tensor.pt')
# ts

In [3]:
# Load adrs tensors from 2_ADR_regressor.ipynb after it's saved (from 2_ADR_regressor_save_tensors.ipynb)
# adrs_ts = torch.load("adr_train_tensors.pt")
# adrs_ts

In [4]:
# note: some of the pre-trained models are freely available but some of them may be gated 
# (possibly still freely available but may require signing up a HF account)
# BERT base transformer model (cased -> case-sensitive) has been used - https://huggingface.co/google-bert/bert-base-cased
# "uncased" version - https://huggingface.co/google-bert/bert-base-uncased

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "abnormal_LFT^^, headache^^, RTI^^, hemoglobin_decreased^^, sperm_count_decreased^^, edema^^, hepatic_cirrhosis(pm), " \
"liver_failure(pm), jaundice(pm), syncope^, sinusitis^, nasal_congestion^, sinus_congestion^, rhinitis^, oropharyngeal_pain^, " \
"epistaxis^, nasopharyngitis^, idiopathic_pulmonary_fibrosis^, anemia^, hematocrit_decreased^, thrombocytopenia(pm), neutropenia(pm), " \
"leukopenia(pm), flushing^, hypotension^, palpitation^, orthostatic_hypotension^, unstable_angina^, hot_flush^, " \
"gastroesophageal_reflux_disease^, diarrhea^, pruritus^, erythema^, angioedema(pm), DRESS(pm), rash(pm), dermatitis(pm), " \
"arthralgia^, joint_swelling^, blurred_vision^, chest_pain^, peripheral_edema^, influenza_like_illness^, vertigo^, fever^, " \
"chest_pain^, hypersensitivity_reaction^, anaphylaxis(pm)"

tokens = tokenizer.tokenize(sequence)
print(tokens)

['abnormal', '_', 'L', '##FT', '^', '^', ',', 'headache', '^', '^', ',', 'R', '##TI', '^', '^', ',', 'hem', '##og', '##lo', '##bin', '_', 'decreased', '^', '^', ',', 'sperm', '_', 'count', '_', 'decreased', '^', '^', ',', 'ed', '##ema', '^', '^', ',', 'he', '##pa', '##tic', '_', 'c', '##ir', '##r', '##hos', '##is', '(', 'pm', ')', ',', 'liver', '_', 'failure', '(', 'pm', ')', ',', 'j', '##au', '##ndi', '##ce', '(', 'pm', ')', ',', 's', '##ync', '##ope', '^', ',', 'sin', '##us', '##itis', '^', ',', 'nasal', '_', 'congestion', '^', ',', 'sin', '##us', '_', 'congestion', '^', ',', 'r', '##hin', '##itis', '^', ',', 'or', '##op', '##har', '##yn', '##ge', '##al', '_', 'pain', '^', ',', 'e', '##pis', '##ta', '##xi', '##s', '^', ',', 'na', '##so', '##pha', '##ryn', '##git', '##is', '^', ',', 'id', '##io', '##pathic', '_', 'pulmonary', '_', 'fi', '##bro', '##sis', '^', ',', 'an', '##emia', '^', ',', 'hem', '##ato', '##c', '##rit', '_', 'decreased', '^', ',', 'th', '##rom', '##bo', '##cy', '##to

In [5]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[22832, 168, 149, 26321, 167, 167, 117, 16320, 167, 167, 117, 155, 21669, 167, 167, 117, 23123, 8032, 2858, 7939, 168, 10558, 167, 167, 117, 20479, 168, 5099, 168, 10558, 167, 167, 117, 5048, 14494, 167, 167, 117, 1119, 4163, 2941, 168, 172, 3161, 1197, 15342, 1548, 113, 9852, 114, 117, 11911, 168, 4290, 113, 9852, 114, 117, 179, 3984, 12090, 2093, 113, 9852, 114, 117, 188, 27250, 15622, 167, 117, 11850, 1361, 10721, 167, 117, 21447, 168, 22860, 167, 117, 11850, 1361, 168, 22860, 167, 117, 187, 8265, 10721, 167, 117, 1137, 4184, 7111, 5730, 2176, 1348, 168, 2489, 167, 117, 174, 19093, 1777, 8745, 1116, 167, 117, 9468, 7301, 20695, 15023, 24632, 1548, 167, 117, 25021, 2660, 21745, 168, 26600, 168, 20497, 12725, 4863, 167, 117, 1126, 20504, 167, 117, 23123, 10024, 1665, 7729, 168, 10558, 167, 117, 24438, 16071, 4043, 3457, 9870, 23179, 113, 9852, 114, 117, 24928, 3818, 12736, 23179, 113, 9852, 114, 117, 5837, 7563, 15622, 5813, 113, 9852, 114, 117, 14991, 1158, 167, 117, 177, 1183, 11439

In [6]:
# convert_ids_to_tokens() - converts numerical IDs back into corresponding token identifiers
token_words = tokenizer.convert_ids_to_tokens(ids)
print(token_words)

['abnormal', '_', 'L', '##FT', '^', '^', ',', 'headache', '^', '^', ',', 'R', '##TI', '^', '^', ',', 'hem', '##og', '##lo', '##bin', '_', 'decreased', '^', '^', ',', 'sperm', '_', 'count', '_', 'decreased', '^', '^', ',', 'ed', '##ema', '^', '^', ',', 'he', '##pa', '##tic', '_', 'c', '##ir', '##r', '##hos', '##is', '(', 'pm', ')', ',', 'liver', '_', 'failure', '(', 'pm', ')', ',', 'j', '##au', '##ndi', '##ce', '(', 'pm', ')', ',', 's', '##ync', '##ope', '^', ',', 'sin', '##us', '##itis', '^', ',', 'nasal', '_', 'congestion', '^', ',', 'sin', '##us', '_', 'congestion', '^', ',', 'r', '##hin', '##itis', '^', ',', 'or', '##op', '##har', '##yn', '##ge', '##al', '_', 'pain', '^', ',', 'e', '##pis', '##ta', '##xi', '##s', '^', ',', 'na', '##so', '##pha', '##ryn', '##git', '##is', '^', ',', 'id', '##io', '##pathic', '_', 'pulmonary', '_', 'fi', '##bro', '##sis', '^', ',', 'an', '##emia', '^', ',', 'hem', '##ato', '##c', '##rit', '_', 'decreased', '^', ',', 'th', '##rom', '##bo', '##cy', '##to

In [7]:
# convert_tokens_to_string() - merges sub-word tokens into complete words
adrs_words = tokenizer.convert_tokens_to_string(tokens)
adrs_words

'abnormal _ LFT ^ ^, headache ^ ^, RTI ^ ^, hemoglobin _ decreased ^ ^, sperm _ count _ decreased ^ ^, edema ^ ^, hepatic _ cirrhosis ( pm ), liver _ failure ( pm ), jaundice ( pm ), syncope ^, sinusitis ^, nasal _ congestion ^, sinus _ congestion ^, rhinitis ^, oropharyngeal _ pain ^, epistaxis ^, nasopharyngitis ^, idiopathic _ pulmonary _ fibrosis ^, anemia ^, hematocrit _ decreased ^, thrombocytopenia ( pm ), neutropenia ( pm ), leukopenia ( pm ), flushing ^, hypotension ^, palpitation ^, orthostatic _ hypotension ^, unstable _ angina ^, hot _ flush ^, gastroesophageal _ reflux _ disease ^, diarrhea ^, pruritus ^, erythema ^, angioedema ( pm ), DRESS ( pm ), rash ( pm ), dermatitis ( pm ), arthralgia ^, joint _ swelling ^, blurred _ vision ^, chest _ pain ^, peripheral _ edema ^, influenza _ like _ illness ^, vertigo ^, fever ^, chest _ pain ^, hypersensitivity _ reaction ^, anaphylaxis ( pm )'

In [8]:
# example to obtain ADR terms from vocabulary indices
adrs_terms = tokenizer.decode([22832, 168, 149, 26321, 167, 167, 117, 16320, 167, 167])
print(adrs_terms)

abnormal _ LFT ^ ^, headache ^ ^


In [9]:
# Try converting the token ID outputs into torch tensors so they can be used in a pytorch model later
# transformers models expect multiple lines of string sequences, so likely need to add tensor dimensions and/or paddings later 
# may be applicable to one line string sequence or multiple string sequences 

In [10]:
# API for PreTrainedTokenizerBase class re. parameter on return_tensors 
# https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.__call__.return_tensors
tokenised_inputs = tokenizer(sequence, return_tensors="pt")
tokenised_inputs
# output contains "input_ids" tensors, "token_type_ids" tensors & "attention_mask" tensors

{'input_ids': tensor([[  101, 22832,   168,   149, 26321,   167,   167,   117, 16320,   167,
           167,   117,   155, 21669,   167,   167,   117, 23123,  8032,  2858,
          7939,   168, 10558,   167,   167,   117, 20479,   168,  5099,   168,
         10558,   167,   167,   117,  5048, 14494,   167,   167,   117,  1119,
          4163,  2941,   168,   172,  3161,  1197, 15342,  1548,   113,  9852,
           114,   117, 11911,   168,  4290,   113,  9852,   114,   117,   179,
          3984, 12090,  2093,   113,  9852,   114,   117,   188, 27250, 15622,
           167,   117, 11850,  1361, 10721,   167,   117, 21447,   168, 22860,
           167,   117, 11850,  1361,   168, 22860,   167,   117,   187,  8265,
         10721,   167,   117,  1137,  4184,  7111,  5730,  2176,  1348,   168,
          2489,   167,   117,   174, 19093,  1777,  8745,  1116,   167,   117,
          9468,  7301, 20695, 15023, 24632,  1548,   167,   117, 25021,  2660,
         21745,   168, 26600,   168, 2

In [11]:
# printing out only the "input_ids" tensors
print(tokenised_inputs["input_ids"])

tensor([[  101, 22832,   168,   149, 26321,   167,   167,   117, 16320,   167,
           167,   117,   155, 21669,   167,   167,   117, 23123,  8032,  2858,
          7939,   168, 10558,   167,   167,   117, 20479,   168,  5099,   168,
         10558,   167,   167,   117,  5048, 14494,   167,   167,   117,  1119,
          4163,  2941,   168,   172,  3161,  1197, 15342,  1548,   113,  9852,
           114,   117, 11911,   168,  4290,   113,  9852,   114,   117,   179,
          3984, 12090,  2093,   113,  9852,   114,   117,   188, 27250, 15622,
           167,   117, 11850,  1361, 10721,   167,   117, 21447,   168, 22860,
           167,   117, 11850,  1361,   168, 22860,   167,   117,   187,  8265,
         10721,   167,   117,  1137,  4184,  7111,  5730,  2176,  1348,   168,
          2489,   167,   117,   174, 19093,  1777,  8745,  1116,   167,   117,
          9468,  7301, 20695, 15023, 24632,  1548,   167,   117, 25021,  2660,
         21745,   168, 26600,   168, 20497, 12725,  

In [12]:
# using pytorch directly to create tensors from token IDs
import torch
torch.tensor(ids)

tensor([22832,   168,   149, 26321,   167,   167,   117, 16320,   167,   167,
          117,   155, 21669,   167,   167,   117, 23123,  8032,  2858,  7939,
          168, 10558,   167,   167,   117, 20479,   168,  5099,   168, 10558,
          167,   167,   117,  5048, 14494,   167,   167,   117,  1119,  4163,
         2941,   168,   172,  3161,  1197, 15342,  1548,   113,  9852,   114,
          117, 11911,   168,  4290,   113,  9852,   114,   117,   179,  3984,
        12090,  2093,   113,  9852,   114,   117,   188, 27250, 15622,   167,
          117, 11850,  1361, 10721,   167,   117, 21447,   168, 22860,   167,
          117, 11850,  1361,   168, 22860,   167,   117,   187,  8265, 10721,
          167,   117,  1137,  4184,  7111,  5730,  2176,  1348,   168,  2489,
          167,   117,   174, 19093,  1777,  8745,  1116,   167,   117,  9468,
         7301, 20695, 15023, 24632,  1548,   167,   117, 25021,  2660, 21745,
          168, 26600,   168, 20497, 12725,  4863,   167,   117, 

In [ ]:
# Adding sample checkpoint & model with the tokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Sample multiple sequence data using ADRs of bosentan and carbamazepine
sequence = ["abnormal_LFT^^, headache^^, RTI^^, hemoglobin_decreased^^, sperm_count_decreased^^, edema^^, hepatic_cirrhosis(pm), " \
"liver_failure(pm), jaundice(pm), syncope^, sinusitis^, nasal_congestion^, sinus_congestion^, rhinitis^, oropharyngeal_pain^, " \
"epistaxis^, nasopharyngitis^, idiopathic_pulmonary_fibrosis^, anemia^, hematocrit_decreased^, thrombocytopenia(pm), " \
"neutropenia(pm), leukopenia(pm), flushing^, hypotension^, palpitation^, orthostatic_hypotension^, unstable_angina^, " \
"hot_flush^, gastroesophageal_reflux_disease^, diarrhea^, pruritus^, erythema^, angioedema(pm), DRESS(pm), rash(pm), " \
"dermatitis(pm), arthralgia^, joint_swelling^, blurred_vision^, chest_pain^, peripheral_edema^, influenza_like_illness^, " \
"vertigo^, fever^, chest_pain^, hypersensitivity_reaction^, anaphylaxis(pm)", "constipation^^, leucopenia^^, dizziness^^, " \
"sedation^^, ataxia^^, elevated_GGT^^, allergic_skin_reactions^^, eosinophilia^, thrombocytopenia^, neutropenia^, headache^, " \
"tremor^, elevated_ALP^, pruritus^, paresthesia^, diplopia^, blurred_vision^, hyponatremia^, fluid_retention^, oedema^, "
"weight_gain^, reduced_plasma_osmolarity_(ADH_like_effect)^, vertigo^"]

tokens = tokenizer(sequence, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

# tokens = tokenizer.tokenize(sequence)
# ids = tokenizer.convert_tokens_to_ids(tokens)
# input_ids = torch.tensor(ids)
# print("Input IDs:", input_ids)

In [19]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.2043, -2.7013],
        [ 2.1903, -1.8928]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
# Making a sample batch of token IDs by using the same sequence twice
batched_ids = [ids, ids]
batched_ids

[[19470,
  1035,
  1048,
  6199,
  1034,
  1034,
  1010,
  14978,
  1034,
  1034,
  1010,
  19387,
  2072,
  1034,
  1034,
  1010,
  19610,
  8649,
  4135,
  8428,
  1035,
  10548,
  1034,
  1034,
  1010,
  18047,
  1035,
  4175,
  1035,
  10548,
  1034,
  1034,
  1010,
  3968,
  14545,
  1034,
  1034,
  1010,
  2002,
  24952,
  2278,
  1035,
  25022,
  12171,
  25229,
  1006,
  7610,
  1007,
  1010,
  11290,
  1035,
  4945,
  1006,
  7610,
  1007,
  1010,
  14855,
  8630,
  6610,
  1006,
  7610,
  1007,
  1010,
  26351,
  17635,
  1034,
  1010,
  8254,
  2271,
  13706,
  1034,
  1010,
  19077,
  1035,
  20176,
  1034,
  1010,
  8254,
  2271,
  1035,
  20176,
  1034,
  1010,
  1054,
  20535,
  7315,
  1034,
  1010,
  20298,
  21890,
  18143,
  3351,
  2389,
  1035,
  3255,
  1034,
  1010,
  4958,
  11921,
  9048,
  2015,
  1034,
  1010,
  17235,
  7361,
  8167,
  6038,
  23806,
  2483,
  1034,
  1010,
  8909,
  3695,
  25940,
  1035,
  21908,
  1035,
  10882,
  12618,
  6190,
  1034,
 

In [15]:
input_batched_ids = torch.tensor(batched_ids)
output_batched = model(input_batched_ids)
print("Logits:", output_batched.logits)

Logits: tensor([[ 2.1536, -1.9253],
        [ 2.1536, -1.9253]], grad_fn=<AddmmBackward0>)


In [16]:
# attention masks are used to tell the attention layers (which contextualise each token) in transformer models 
# to ignore the padding tokens when multiple sequences are of different lengths